In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import os
import json

import pandas as pd
from transformers import AutoTokenizer

from rate_severity_of_toxic_comments.utilities import *
from rate_severity_of_toxic_comments.model import *
from rate_severity_of_toxic_comments.dataset import *
from rate_severity_of_toxic_comments.training import * 
from rate_severity_of_toxic_comments.preprocessing import * 

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [33]:
DEFAULT_CONFIG_FILE_PATH = "config/default.json"
LOCAL_CONFIG_FILE_PATH = "config/local.json"


default = open(DEFAULT_CONFIG_FILE_PATH)
CONFIG = json.load(default)

if os.path.exists(LOCAL_CONFIG_FILE_PATH):
    with open(LOCAL_CONFIG_FILE_PATH) as local:
        CONFIG.update(json.load(local))

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])
# CONFIG['group'] = f'{HASH_NAME}-Baseline'

In [34]:
df = pd.read_csv("res/data/weighted_train_small.csv")
# df = pd.read_csv("res/data/validation_data.csv")
df = df.sample(20)
data_size = len(df.index)

In [35]:
train_split = 0.7
threshold_index = int(train_split * data_size)

df_train = df[:threshold_index].reset_index(drop=True)
df_valid = df[threshold_index:].reset_index(drop=True)

In [ ]:
preproc_pipelines = CONFIG['preprocessing']

### JUST FOR TESTING APPLIED ALSO ON VALIDATION SET

for df in [df_train, df_valid]:
    df = apply_preprocessing_pipelines(df, preproc_pipelines)

In [ ]:
training_data, val_data = build_datasets([df_train, df_valid], CONFIG, ["weighted", "weighted"])

In [36]:
stats = run_training(training_data, val_data, log_interval=10, config=CONFIG, verbose=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: toxicity (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
C:\Users\Utente\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([14])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Utente\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1  Lr: 0.00010000  | Time one epoch (s): 13.6047  
 Train -  Loss: [3353495.7500]  
 Val   -  Loss: [9822592.0000] 
Epoch: 2  Lr: 0.00010000  | Time one epoch (s): 12.8176  
 Train -  Loss: [3352563.5000]  
 Val   -  Loss: [9819861.0000] 
Epoch: 3  Lr: 0.00010000  | Time one epoch (s): 12.6170  
 Train -  Loss: [3351630.5000]  
 Val   -  Loss: [9817129.0000] 
Epoch: 4  Lr: 0.00010000  | Time one epoch (s): 12.0133  
 Train -  Loss: [3350698.7500]  
 Val   -  Loss: [9814399.0000] 
Epoch: 5  Lr: 0.00010000  | Time one epoch (s): 12.4276  
 Train -  Loss: [3349766.5000]  
 Val   -  Loss: [9811666.0000] 
Epoch: 6  Lr: 0.00010000  | Time one epoch (s): 12.4227  
 Train -  Loss: [3348834.2500]  
 Val   -  Loss: [9808935.0000] 
Epoch: 7  Lr: 0.00010000  | Time one epoch (s): 12.6462  
 Train -  Loss: [3347902.2500]  
 Val   -  Loss: [9806205.0000] 
Epoch: 8  Lr: 0.00010000  | Time one epoch (s): 12.1918  
 Train -  Loss: [3346970.2500]  
 Val   -  Loss: [9803474.0000] 
Epoch: 9  Lr: 0.

train_loss,█▇▆▆▅▄▃▃▂▁
valid_loss,█▇▆▆▅▄▃▃▂▁
train_loss,3345106.25
valid_loss,9798017.0
